In [3]:
!pip install -q ipywidgets

from IPython.display import display, Markdown, Math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib.patches import Rectangle

# 1) Formulas section
formulas = widgets.VBox([
    widgets.Label("## Formulas Used"),
    widgets.Label(r"$v = \frac{\dot V}{A}$", _dom_classes=['output_latex']),
    widgets.Label(r"$\mathrm{Re} = \frac{\rho v D_i}{\mu}$", _dom_classes=['output_latex']),
    widgets.Label(r"""$\displaystyle f =
    \begin{cases}64/\mathrm{Re}, & \mathrm{Re}<2300,\\
    0.3164\,\mathrm{Re}^{-0.25}, & \text{otherwise}\end{cases}$""", _dom_classes=['output_latex']),
    widgets.Label(r"$\Delta P = f \frac{L}{D_i} \frac12 \rho v^2$", _dom_classes=['output_latex']),
    widgets.Label(r"$R_{int} = 1/(h_{int}P_sL), R_{pipe} = \ln(r_o/r_i)/(2\pi k_{pipe}L)$", _dom_classes=['output_latex']),
    widgets.Label(r"$R_{ins} = \ln(r_{o,ins}/r_o)/(2\pi k_{ins}L), R_{ext} = 1/(h_{ext}P_sL)$", _dom_classes=['output_latex']),
    widgets.Label(r"$U = 1/(R_{int}+R_{pipe}+R_{ins}+R_{ext})$", _dom_classes=['output_latex']),
    widgets.Label(r"$Q = U P_s L (T_{in}-T_{amb})$", _dom_classes=['output_latex']),
    widgets.Label(r"$T_{out} = T_{in} - Q/(\dot m c_p)$", _dom_classes=['output_latex']),
    widgets.Label(r"$T_{fl}(x) = T_{amb} + (T_{in}-T_{amb})e^{-U P_s x/(\dot m c_p)}$", _dom_classes=['output_latex']),
    widgets.Label(r"$q'(x) = U(T_{fl}(x)-T_{amb})$", _dom_classes=['output_latex'])
])

# 2) Material data
pipe_materials = {'Steel':45,'Copper':390,'PVC':0.19,'Aluminum':205,'Stainless Steel':16,'Brass':109}
ins_materials  = {'Fiberglass':0.04,'Mineral Wool':0.035,'Foam':0.025,'Rockwool':0.037,'Aerogel':0.013}
coolant_materials = {
    'Water':{'rho':1000,'mu':0.001,'k':0.6,'cp':4181},
    'Ethylene Glycol':{'rho':1110,'mu':0.02,'k':0.25,'cp':2400},
    'Oil':{'rho':850,'mu':0.1,'k':0.13,'cp':2000},
    'Ammonia':{'rho':682,'mu':0.0006,'k':0.51,'cp':2008},
    'Methanol':{'rho':792,'mu':0.0006,'k':0.2,'cp':2500}
}

# 3) Input widgets with non-negative bounds
pipe_dd    = widgets.Dropdown(options=list(pipe_materials), description='Pipe:')
ins_dd     = widgets.Dropdown(options=list(ins_materials),   description='Insulation:')
cool_dd    = widgets.Dropdown(options=list(coolant_materials), description='Coolant:')
flow_txt   = widgets.BoundedFloatText(value=0.1, min=0,     description='Flow (m³/s):')
T_in_txt   = widgets.BoundedFloatText(value=20, min=-273.15, description='Inlet Temp (°C):')
T_amb_txt  = widgets.BoundedFloatText(value=25, min=-273.15, description='Ambient Temp (°C):')
L_txt      = widgets.BoundedFloatText(value=10, min=0,      description='Length (m):')
Di_txt     = widgets.BoundedFloatText(value=0.05, min=0,    description='Inner Dia (m):')
t_pipe_txt = widgets.BoundedFloatText(value=0.005, min=0,   description='Pipe Thick (m):')
t_ins_txt  = widgets.BoundedFloatText(value=0.02, min=0,    description='Insul Thick (m):')
h_int_txt  = widgets.BoundedFloatText(value=1000, min=0,    description='h_int (W/m²K):')
h_ext_txt  = widgets.BoundedFloatText(value=10, min=0,      description='h_ext (W/m²K):')

inputs = widgets.VBox([
    widgets.HBox([pipe_dd, ins_dd, cool_dd]),
    widgets.HBox([flow_txt, T_in_txt, T_amb_txt]),
    widgets.HBox([L_txt, Di_txt, t_pipe_txt]),
    widgets.HBox([t_ins_txt, h_int_txt, h_ext_txt])
])

# 4) Outputs
graphs_out = widgets.Output()
results_out = widgets.Output()

# 5) Update function
def update(*args):
    # read
    pipe, ins, cool = pipe_dd.value, ins_dd.value, cool_dd.value
    flow, T_in, T_amb = flow_txt.value, T_in_txt.value, T_amb_txt.value
    L, Di, t_pipe, t_ins = L_txt.value, Di_txt.value, t_pipe_txt.value, t_ins_txt.value
    h_int, h_ext = h_int_txt.value, h_ext_txt.value
    rho = coolant_materials[cool]['rho']; mu = coolant_materials[cool]['mu']; cp = coolant_materials[cool]['cp']
    k_pipe, k_ins = pipe_materials[pipe], ins_materials[ins]
    m_dot = rho*flow
    r_i, r_o_p, r_o_ins = Di/2, Di/2+t_pipe, Di/2+t_pipe+t_ins
    A, P_s = np.pi*r_i**2, 2*np.pi*r_o_ins
    v = flow/A; Re = rho*v*Di/mu
    f = 64/Re if Re<2300 else 0.3164*Re**(-0.25)
    dP_tot = f*(L/Di)*0.5*rho*v**2
    R_int = 1/(h_int*P_s*L)
    R_pipe = np.log(r_o_p/r_i)/(2*np.pi*k_pipe*L)
    R_ins = np.log(r_o_ins/r_o_p)/(2*np.pi*k_ins*L)
    R_ext = 1/(h_ext*P_s*L)
    U_val = 1/(R_int+R_pipe+R_ins+R_ext)
    Q_val = U_val*P_s*L*(T_in-T_amb)
    T_out = T_in - Q_val/(m_dot*cp)
    x = np.linspace(0, L, 300)
    P_prof = dP_tot*x/L
    T_fl = T_amb+(T_in-T_amb)*np.exp(-U_val*P_s*x/(m_dot*cp))
    q_flux = U_val*(T_fl-T_amb)
    Q_cum = m_dot*cp*(T_in - T_fl)

    # graphs
    with graphs_out:
        graphs_out.clear_output()
        fig, axs = plt.subplots(2,2,figsize=(16,10),dpi=100)
        # schematic
        axs[0,0].add_patch(Rectangle((0,0),L,2*t_ins+2*t_pipe+Di,color='lightgreen',label='Insulation'))
        axs[0,0].add_patch(Rectangle((0,t_ins),L,Di+2*t_pipe,color='silver',label='Pipe'))
        axs[0,0].add_patch(Rectangle((0,t_ins+t_pipe),L,Di,color='skyblue',label='Coolant'))
        axs[0,0].legend(loc='upper right'); axs[0,0].axis('off')
        axs[0,0].set_title(f"Schematic (Coolant={cool}, Ins={ins})")
        # ΔP
        axs[0,1].plot(x,P_prof,linewidth=2); axs[0,1].set_title('ΔP vs Distance'); axs[0,1].set(xlabel='x (m)',ylabel='ΔP (Pa)'); axs[0,1].grid(True)
        # Temp
        axs[1,0].plot(x,T_fl,linewidth=2,label='Fluid T'); axs[1,0].plot(x,T_amb+q_flux/h_ext,linewidth=2,linestyle='--',label='Surface T')
        axs[1,0].set_title('Temp vs Distance'); axs[1,0].set(xlabel='x (m)',ylabel='T (°C)'); axs[1,0].legend(); axs[1,0].grid(True)
        # q flux
        axs[1,1].plot(x,q_flux,linewidth=2,color='firebrick'); axs[1,1].set_title("Local Heat Flux vs Distance")
        axs[1,1].set(xlabel='x (m)',ylabel="q' (W/m²)"); axs[1,1].grid(True)
        plt.tight_layout(); plt.show()
        # cumulative Q
        plt.figure(figsize=(8,4),dpi=100)
        plt.plot(x,Q_cum,linewidth=2,color='teal'); plt.title('Cumulative Q vs Distance'); plt.xlabel('x (m)'); plt.ylabel('Q(x) (W)'); plt.grid(True); plt.tight_layout(); plt.show()

    # results
    with results_out:
        results_out.clear_output()
        res = {
            'Velocity (m/s)':v, 'Reynolds':Re, 'Friction':f,
            'ΔP_total (Pa)':dP_tot, 'Gradient (Pa/m)':dP_tot/L,
            'Mass Flow (kg/s)':m_dot, 'Overall U':U_val,
            'Q (W)':Q_val, 'Q/m (W/m)':Q_val/L, 'T_out (°C)':T_out
        }
        df = pd.DataFrame.from_dict(res,orient='index',columns=['Value'])
        display(Markdown("## Results Summary"))
        display(Markdown(df.to_markdown()))

# observe changes
for widget in [pipe_dd, ins_dd, cool_dd, flow_txt, T_in_txt, T_amb_txt, L_txt, Di_txt, t_pipe_txt, t_ins_txt, h_int_txt, h_ext_txt]:
    widget.observe(update,'value')

update()  # initial

# Accordion to section
acc = widgets.Accordion(children=[formulas, inputs, graphs_out, results_out])
titles = ['Formulas','Initial Conditions','Graphs','Results']
for i, t in enumerate(titles): acc.set_title(i,t)

display(Markdown("# 1D Pipe Heat & Pressure Calculator"))
display(acc)

# 1D Pipe Heat & Pressure Calculator

Accordion(children=(VBox(children=(Label(value='## Formulas Used'), Label(value='$v = \\frac{\\dot V}{A}$', _d…